In [135]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [181]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [182]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [183]:
# one review
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

## Text Cleaning

1. Sample 10000 rows
2. Remove html tags
3. Remove special characteers
4. Stemming (play , playing  , played are all same words so we convert all these words  to same word)
5. Removing Stop words


In [184]:
df.shape

(50000, 2)

In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [186]:
df['sentiment'].replace({'positive':1, 'negative':0}, inplace = True)

In [187]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [188]:
import re
clean = re.compile('<.*?>')
re.sub(clean, '' , df.iloc[2].review)

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [189]:
# function to clean html tag
def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [190]:
df['review'] = df['review'].apply(clean_html)

In [191]:
df['review'][3310]

'I heard legends about this "film" (quotes used so as not to insult films) for a while, so when I finally got the DVD with it, I impatiently started watching it. By the end, I *had* to fast forward through just a few of the most moronic, ineptly made, nonsensical scenes of this pointless childish mess to make it end quicker.This may be the worst film I\'ve even "touched" - and I used to be associated with Troma for a while. "Manson Family" makes the bottom of Troma\'s entries look like daring and groundbreaking art-house filmwork. I could go on and talk about the syphilitic skeleton of a "plot" it has, the revoltingly bad "acting", the painful, inept "directing", the sets and props with their "dollar ninety nine" look (I especially "loved" the plastic toy guns used in the Tate murder scene!) or the nauseating look and feel of this whole bag of garbage (I think it was supposed to represent a drug-induced hallucination; I have absolutely no idea how a drug-induced hallucination looks or 

In [192]:
def convert_lower(text):
    return text.lower()

In [193]:
df['review'] = df['review'].apply(convert_lower)

In [194]:
df['review'][3310]

'i heard legends about this "film" (quotes used so as not to insult films) for a while, so when i finally got the dvd with it, i impatiently started watching it. by the end, i *had* to fast forward through just a few of the most moronic, ineptly made, nonsensical scenes of this pointless childish mess to make it end quicker.this may be the worst film i\'ve even "touched" - and i used to be associated with troma for a while. "manson family" makes the bottom of troma\'s entries look like daring and groundbreaking art-house filmwork. i could go on and talk about the syphilitic skeleton of a "plot" it has, the revoltingly bad "acting", the painful, inept "directing", the sets and props with their "dollar ninety nine" look (i especially "loved" the plastic toy guns used in the tate murder scene!) or the nauseating look and feel of this whole bag of garbage (i think it was supposed to represent a drug-induced hallucination; i have absolutely no idea how a drug-induced hallucination looks or 

In [195]:
# function to remove special character
def remove_special(text):
    x=''
    
    for i in text:
        if i.isalnum():
            x = x+i
        else:
            x=x+ ' '
    return x

In [196]:
df['review'] = df['review'].apply(remove_special)

In [197]:
df['review'][3310]

'i heard legends about this  film   quotes used so as not to insult films  for a while  so when i finally got the dvd with it  i impatiently started watching it  by the end  i  had  to fast forward through just a few of the most moronic  ineptly made  nonsensical scenes of this pointless childish mess to make it end quicker this may be the worst film i ve even  touched    and i used to be associated with troma for a while   manson family  makes the bottom of troma s entries look like daring and groundbreaking art house filmwork  i could go on and talk about the syphilitic skeleton of a  plot  it has  the revoltingly bad  acting   the painful  inept  directing   the sets and props with their  dollar ninety nine  look  i especially  loved  the plastic toy guns used in the tate murder scene   or the nauseating look and feel of this whole bag of garbage  i think it was supposed to represent a drug induced hallucination  i have absolutely no idea how a drug induced hallucination looks or fe

In [198]:
# remove the stop words
import nltk

from nltk.corpus import stopwords

In [199]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [200]:
def remove_stopwords(text):
    x = []
    for i in text.split():
        x.append(i)
    y = x[:]
    x.clear()
    return y
       

In [201]:
df['review'] = df['review'].apply(remove_stopwords)

In [202]:
df

,review,sentiment
0,"[one, of, the, other, reviewers, has, mentione...",1
1,"[a, wonderful, little, production, the, filmin...",1
2,"[i, thought, this, was, a, wonderful, way, to,...",1
3,"[basically, there, s, a, family, where, a, lit...",0
4,"[petter, mattei, s, love, in, the, time, of, m...",1
...,...,...
49995,"[i, thought, this, movie, did, a, down, right,...",1
49996,"[bad, plot, bad, dialogue, bad, acting, idioti...",0
49997,"[i, am, a, catholic, taught, in, parochial, el...",0
49998,"[i, m, going, to, have, to, disagree, with, th...",0


In [203]:
# perform stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [204]:
y = []
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z = y[:]
    y.clear()
    return z

In [205]:
stem_words(['i','loved','loving','it'])

['i', 'love', 'love', 'it']

In [206]:
df['review'] = df['review'].apply(stem_words)

In [207]:
df

,review,sentiment
0,"[one, of, the, other, review, ha, mention, tha...",1
1,"[a, wonder, littl, product, the, film, techniq...",1
2,"[i, thought, thi, wa, a, wonder, way, to, spen...",1
3,"[basic, there, s, a, famili, where, a, littl, ...",0
4,"[petter, mattei, s, love, in, the, time, of, m...",1
...,...,...
49995,"[i, thought, thi, movi, did, a, down, right, g...",1
49996,"[bad, plot, bad, dialogu, bad, act, idiot, dir...",0
49997,"[i, am, a, cathol, taught, in, parochi, elemen...",0
49998,"[i, m, go, to, have, to, disagre, with, the, p...",0


In [208]:
# join back
def join_back(list_input):
    return " ".join(list_input)

In [209]:
df['review'] = df['review'].apply(join_back)

In [210]:
df

,review,sentiment
0,one of the other review ha mention that after ...,1
1,a wonder littl product the film techniqu is ve...,1
2,i thought thi wa a wonder way to spend time on...,1
3,basic there s a famili where a littl boy jake ...,0
4,petter mattei s love in the time of money is a...,1
...,...,...
49995,i thought thi movi did a down right good job i...,1
49996,bad plot bad dialogu bad act idiot direct the ...,0
49997,i am a cathol taught in parochi elementari sch...,0
49998,i m go to have to disagre with the previou com...,0


In [211]:
X = df.iloc[:,0:1].values

In [212]:
X.shape

(50000, 1)

In [213]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features =1000)


In [214]:
X = cv.fit_transform(df['review']).toarray()

In [215]:
X.shape

(50000, 1000)

In [216]:
 X[0].max()

16

In [217]:
y = df.iloc[:,-1].values

In [218]:
y.shape

(50000,)

In [219]:
# X,y
#Trainig set  
# Test test


In [220]:
from sklearn.model_selection import train_test_split

In [221]:
X_train , X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [222]:
X_train.shape

(40000, 1000)

In [223]:
X_test.shape

(10000, 1000)

In [224]:
y_train.shape

(40000,)

In [225]:
y_test.shape

(10000,)

In [226]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB , BernoulliNB

In [227]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()

In [228]:
clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)

BernoulliNB()

In [229]:
y_predict1 = clf1.predict(X_test)
y_predict2 = clf2.predict(X_test)
y_predict3 = clf3.predict(X_test)

In [230]:
y_test.shape

(10000,)

In [231]:
y_predict1.shape

(10000,)

In [232]:
from sklearn.metrics import accuracy_score

In [233]:
print("Gaussian" , accuracy_score(y_test, y_predict1))
print("Multinomial" ,accuracy_score(y_test, y_predict2))
print("Bernauli" ,accuracy_score(y_test, y_predict3))

Gaussian 0.8068
Multinomial 0.8175
Bernauli 0.8241
